In [19]:
import numpy as np
import cv2
import glob
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import keras
import openpyxl

In [20]:
def transfer_learning_Model(model='VGG16'):
    # Add a GlobalAveragePooling2D layer and a Dense layer on top of the pre-trained model
    if model=='VGG16':
        base_model = VGG16(input_shape=(img_size[0],img_size[0], 3), include_top=False, weights='imagenet')
        x = Flatten()(base_model.output)
    elif model=='MobileNetV2':
        base_model = MobileNetV2(input_shape=(img_size[0],img_size[0], 3),weights='imagenet', include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
    elif model=='InceptionV3':
        base_model = InceptionV3(input_shape=(img_size[0],img_size[0], 3),weights='imagenet', include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
    elif model=='DenseNet121':
        base_model = DenseNet121(input_shape=(img_size[0],img_size[0], 3),weights='imagenet', include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
    elif model=='ResNet50':
        base_model = ResNet50(input_shape=(img_size[0],img_size[0], 3),weights='imagenet', include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation='relu')(x)
    x = Dense(NUM_CLASSES, activation='softmax')(x)

    # Freeze the layers in the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False


    model = Model(inputs=base_model.input, outputs=x)
    return model

In [21]:
# Define the path to the two folders containing the image files
folder1_path = "/kaggle/input/fundus-images/Data_Processed_Gray/Diseased/"
folder2_path = "/kaggle/input/fundus-images/Data_Processed_Gray/Healthy/"


# Define the image size and number of channels
img_size = (224, 224)
num_channels = 3
NUM_CLASSES=2
BATCH_SIZE=32
EPOCHS=1

model_name='VGG16'

n_splits = 2

In [27]:
# Initialize lists to store image data and labels
X = []
y = []

# Load image files from folder1 and append to X and y lists
for filepath in glob.glob(folder1_path + "*.jpg"):
    img = cv2.imread(filepath)
    img = cv2.resize(img, img_size)
    X.append(img/255.)
    y.append(0) # Set label to 0 for images in folder1

# Load image files from folder2 and append to X and y lists
for filepath in glob.glob(folder2_path + "*.jpg"):
    img = cv2.imread(filepath)
    img = cv2.resize(img, img_size)
    X.append(img/255.)
    y.append(1) # Set label to 1 for images in folder2

# Convert X and y to numpy arrays
X = np.array(X)
y = np.array(y)

print('Loading Complete')
# Define the VGG16 model
model = transfer_learning_Model(model=model_name)


# Define the data augmentation generator
data_augmentation = ImageDataGenerator(
        rotation_range=10,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
)


# Initialize lists to store evaluation metrics for each fold
accuracies = []
precisions = []
f1scores = []
recalls = []

# Initialize a KFold object
kf = KFold(n_splits=n_splits, shuffle=True)

# Perform cross-validation
fold = 1
for train_index, test_index in kf.split(X):
    print(f"Fold {fold}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train your model on the training data
    optimizer = Adam(lr=0.0001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    # Calculate class weights based on the training data
    class_weights = {0: len(y_train[y_train== 0])/len(y_train), 
                     1: len(y_train[y_train == 1])/len(y_train)}
    
    print(class_weights)
    # Create a checkpoint to save the best model weights
    checkpoint_path = f'fold{fold}-{model_name}-best.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
    
    # Fit the model with data augmentation and the checkpoint
    history = model.fit(
        data_augmentation.flow(X_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,
        validation_data=(X_test, y_test),
        callbacks=[checkpoint]
    )
    
    model = keras.models.load_model(f'fold{fold}-{model_name}-best.h5')

    # Make predictions on the test data using the trained model
    y_pred = model.predict(X_test)

    # Evaluate the predictions
    y_pred=np.argmax(y_pred,axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    f1score = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Print the evaluation metrics
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"F1 Score: {f1score}")
    print(f"Recall: {recall}")

    # Append the evaluation metrics to the lists
    accuracies.append(accuracy)
    precisions.append(precision)
    f1scores.append(f1score)
    recalls.append(recall)
    print('----------------------------------------------')
    print(f'fold-{fold}-completed')
    print('----------------------------------------------')
    fold=fold+1



Loading Complete
Fold 1
{0: 0.7397260273972602, 1: 0.2602739726027397}
4/4 [==============================] - ETA: 0s - loss: 3.0494 - accuracy: 0.6667
Epoch 1: val_accuracy improved from -inf to 0.71233, saving model to fold1-VGG16-best.h5
5/5 [==============================] - 0s 62ms/step


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7123287671232876
Precision: 0.507412272471383
F1 Score: 0.5926575342465753
Recall: 0.7123287671232876
----------------------------------------------
fold-1-completed
----------------------------------------------
Fold 2
{0: 0.7123287671232876, 1: 0.2876712328767123}
4/4 [==============================] - ETA: 0s - loss: 3.6506 - accuracy: 0.5000
Epoch 1: val_accuracy improved from -inf to 0.73973, saving model to fold2-VGG16-best.h5
5/5 [==============================] - 0s 66ms/step
Accuracy: 0.7397260273972602
Precision: 0.5471945956089321
F1 Score: 0.6290583540071191
Recall: 0.7397260273972602
----------------------------------------------
fold-2-completed
----------------------------------------------


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
# Write the evaluation metrics to a text file
with open(f'evaluation_metrics_Res_{model_name}.txt', 'w') as f:
    f.write(f"Accuracy: {accuracies}\n")
    f.write(f"Precision: {precisions}\n")
    f.write(f"F1 Score: {f1scores}\n")
    f.write(f"Recall: {recalls}\n")

In [31]:
# Create a new workbook
workbook = openpyxl.Workbook()

# Create a new sheet
sheet = workbook.active

# Write the header row
header_row = ["Fold", "Accuracy", "Precision", "Recall", "F1 Score"]
sheet.append(header_row)

# Write the evaluation metrics for each fold
for fold in range(fold):
    row = [fold+1, accuracies[fold], precisions[fold], recalls[fold], f1scores[fold]]
    sheet.append(row)

# Save the workbook
workbook.save(f"evaluation_{model_name}.xlsx")
workbook.close()